In [1]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings("ignore")

In [2]:
swm = pd.read_csv('swm_infrastructure.csv')
ses = pd.read_csv('ses_predictors_v2.csv')
lu = pd.read_csv('landuse_predictors_v3.csv')
df311 = pd.read_csv('311_cleaned.csv')

## Overview

In [3]:
swm.head()

,Unnamed: 0,cd,boro,date,year,month,REFUSETONSCOLLECTED,PAPERTONSCOLLECTED,MGPTONSCOLLECTED,RESORGANICSTONS,...,shape_area,wfr_avg_recycling,wfr_avg_refuse,wfr_avg_bulk,bigbelly,seasonal_wire_litter_basket,highend_basket,publicspacereceptacle,wire_basket,total_litter_baskets
0,0,BK01,BK,2016-11-01,2016,11,5250.4,527.8,402.8,16.9,...,0.001307,1.0,3.0,3.0,1.0,0.0,2.0,1.0,10.0,14.0
1,1,BK01,BK,2016-12-01,2016,12,5409.5,726.3,501.0,11.7,...,0.001307,1.0,3.0,3.0,1.0,0.0,2.0,1.0,10.0,14.0
2,2,BK01,BK,2017-01-01,2017,1,5189.9,589.1,455.2,14.9,...,0.001307,1.0,3.0,3.0,1.0,0.0,2.0,1.0,10.0,14.0
3,3,BK01,BK,2017-02-01,2017,2,4705.4,505.6,395.8,10.5,...,0.001307,1.0,3.0,3.0,1.0,0.0,2.0,1.0,10.0,14.0
4,4,BK01,BK,2017-03-01,2017,3,5476.7,563.0,453.3,10.6,...,0.001307,1.0,3.0,3.0,1.0,0.0,2.0,1.0,10.0,14.0


In [4]:
swm.shape

(3540, 28)

In [5]:
ses.head()

,Unnamed: 0,Unnamed: 0.1,borocd,pct_bach_deg,over65_rate,under18_rate,pct_white_nh,poverty_rate,pct_hh_rent_burd,mean_commute,unemployment,crime_per_1000,pop_density,month,borough_no,cd,borough_code,community_district
0,0,0,201,11.3,9.0,29.7,2.1,32.3,49.4,42.5,6.4,21.7,42230.049029,2016-11-01,2,1,BX,BX01
1,1,1,202,14.2,9.2,27.5,1.9,32.3,53.8,44.9,6.4,12.3,23547.039187,2016-11-01,2,2,BX,BX02
2,2,2,203,14.1,9.2,29.1,1.2,31.4,48.0,43.9,6.6,13.6,49616.354945,2016-11-01,2,3,BX,BX03
3,3,3,206,12.6,9.6,28.3,6.1,31.4,55.1,44.4,8.4,12.0,54409.272313,2016-11-01,2,6,BX,BX06
4,4,4,204,15.8,9.7,27.7,1.9,34.7,54.0,41.9,6.4,14.6,73513.072409,2016-11-01,2,4,BX,BX04


In [6]:
ses.shape

(3540, 18)

In [7]:
lu.head()

,month,community_district,numbldgs,unitstotal,asland_sqft,pct_clean_strts,pct_lot_area_open_space,pct_lot_area_public_facility_institution,pct_lot_area_res_1_2_family_bldg,pct_lot_area_res_multifamily_walkup,restaurant,business
0,2016-11-01,MN01,1760,55988,79.282161,100.0,4.35,31.68,0.17,0.63,3295,683
1,2016-11-01,MN02,5375,72165,100.324753,98.7,3.55,9.63,4.47,10.26,1271,1013
2,2016-11-01,MN03,4708,94059,58.236802,93.4,14.95,11.15,0.32,8.66,580,947
3,2016-11-01,MN04,3923,100090,105.402387,100.0,2.81,11.75,0.88,7.68,964,1014
4,2016-11-01,MN05,3190,95425,516.066825,100.0,1.59,6.49,0.09,0.67,270,2083


In [8]:
lu.shape

(3540, 12)

In [9]:
df311.head()

,Unnamed: 0,CD,Year_Month,Year,Month,Count
0,0,BK01,2016-11,2016,11,50
1,1,BK01,2016-12,2016,12,36
2,2,BK01,2017-01,2017,1,51
3,3,BK01,2017-02,2017,2,61
4,4,BK01,2017-03,2017,3,67


In [10]:
df311.shape

(3523, 6)

## Cleaning

### swm

In [11]:
swm.columns

Index(['Unnamed: 0', 'cd', 'boro', 'date', 'year', 'month',
       'REFUSETONSCOLLECTED', 'PAPERTONSCOLLECTED', 'MGPTONSCOLLECTED',
       'RESORGANICSTONS', 'fs_openhours', 'fs_open', 'trs_permits',
       'paper_baskets', 'mgp_baskets', 'total_number_of_households',
       'hh_compost_num', 'hh_compost_percent', 'shape_area',
       'wfr_avg_recycling', 'wfr_avg_refuse', 'wfr_avg_bulk', 'bigbelly',
       'seasonal_wire_litter_basket', 'highend_basket',
       'publicspacereceptacle', 'wire_basket', 'total_litter_baskets'],
      dtype='object')

In [12]:
swm['Year_Month'] = pd.to_datetime(swm['date']).dt.to_period('M')
swm = swm.drop(['Unnamed: 0','boro','date'],axis=1)

In [13]:
swm['Year_Month'].unique()

<PeriodArray>
['2016-11', '2016-12', '2017-01', '2017-02', '2017-03', '2017-04', '2017-05',
 '2017-06', '2017-07', '2017-08', '2017-09', '2017-10', '2017-11', '2017-12',
 '2018-01', '2018-02', '2018-03', '2018-04', '2018-05', '2018-06', '2018-07',
 '2018-08', '2018-09', '2018-10', '2018-11', '2018-12', '2019-01', '2019-02',
 '2019-03', '2019-04', '2019-05', '2019-06', '2019-07', '2019-08', '2019-09',
 '2019-10', '2019-11', '2019-12', '2020-01', '2020-02', '2020-03', '2020-04',
 '2020-05', '2020-06', '2020-07', '2020-08', '2020-09', '2020-10', '2020-11',
 '2020-12', '2021-01', '2021-02', '2021-03', '2021-04', '2021-05', '2021-06',
 '2021-07', '2021-08', '2021-09', '2021-10']
Length: 60, dtype: period[M]

In [14]:
swm.dtypes

cd                                object
year                               int64
month                              int64
REFUSETONSCOLLECTED              float64
PAPERTONSCOLLECTED               float64
MGPTONSCOLLECTED                 float64
RESORGANICSTONS                  float64
fs_openhours                     float64
fs_open                          float64
trs_permits                      float64
paper_baskets                    float64
mgp_baskets                      float64
total_number_of_households         int64
hh_compost_num                     int64
hh_compost_percent               float64
shape_area                       float64
wfr_avg_recycling                float64
wfr_avg_refuse                   float64
wfr_avg_bulk                     float64
bigbelly                         float64
seasonal_wire_litter_basket      float64
highend_basket                   float64
publicspacereceptacle            float64
wire_basket                      float64
total_litter_bas

In [15]:
swm.rename({'cd':'CD','year':'Year','month':'Month'},inplace=True,axis=1)
swm.head()

,CD,Year,Month,REFUSETONSCOLLECTED,PAPERTONSCOLLECTED,MGPTONSCOLLECTED,RESORGANICSTONS,fs_openhours,fs_open,trs_permits,...,wfr_avg_recycling,wfr_avg_refuse,wfr_avg_bulk,bigbelly,seasonal_wire_litter_basket,highend_basket,publicspacereceptacle,wire_basket,total_litter_baskets,Year_Month
0,BK01,2016,11,5250.4,527.8,402.8,16.9,333.25,6.0,14.0,...,1.0,3.0,3.0,1.0,0.0,2.0,1.0,10.0,14.0,2016-11
1,BK01,2016,12,5409.5,726.3,501.0,11.7,333.25,6.0,14.0,...,1.0,3.0,3.0,1.0,0.0,2.0,1.0,10.0,14.0,2016-12
2,BK01,2017,1,5189.9,589.1,455.2,14.9,333.25,6.0,14.0,...,1.0,3.0,3.0,1.0,0.0,2.0,1.0,10.0,14.0,2017-01
3,BK01,2017,2,4705.4,505.6,395.8,10.5,333.25,6.0,14.0,...,1.0,3.0,3.0,1.0,0.0,2.0,1.0,10.0,14.0,2017-02
4,BK01,2017,3,5476.7,563.0,453.3,10.6,333.25,6.0,14.0,...,1.0,3.0,3.0,1.0,0.0,2.0,1.0,10.0,14.0,2017-03


### ses

In [16]:
ses['Year_Month'] = pd.to_datetime(ses['month']).dt.to_period('M')

In [17]:
ses = ses.drop(['Unnamed: 0','Unnamed: 0.1','borocd','borough_no','cd','borough_code'],axis=1)

In [18]:
ses.rename({'community_district':'CD'},inplace=True,axis=1)
ses.head()

,pct_bach_deg,over65_rate,under18_rate,pct_white_nh,poverty_rate,pct_hh_rent_burd,mean_commute,unemployment,crime_per_1000,pop_density,month,CD,Year_Month
0,11.3,9.0,29.7,2.1,32.3,49.4,42.5,6.4,21.7,42230.049029,2016-11-01,BX01,2016-11
1,14.2,9.2,27.5,1.9,32.3,53.8,44.9,6.4,12.3,23547.039187,2016-11-01,BX02,2016-11
2,14.1,9.2,29.1,1.2,31.4,48.0,43.9,6.6,13.6,49616.354945,2016-11-01,BX03,2016-11
3,12.6,9.6,28.3,6.1,31.4,55.1,44.4,8.4,12.0,54409.272313,2016-11-01,BX06,2016-11
4,15.8,9.7,27.7,1.9,34.7,54.0,41.9,6.4,14.6,73513.072409,2016-11-01,BX04,2016-11


### lu

In [19]:
lu['Year_Month'] = pd.to_datetime(ses['month']).dt.to_period('M')

In [20]:
lu.rename({'community_district':'CD'},inplace=True,axis=1)
lu.head()

,month,CD,numbldgs,unitstotal,asland_sqft,pct_clean_strts,pct_lot_area_open_space,pct_lot_area_public_facility_institution,pct_lot_area_res_1_2_family_bldg,pct_lot_area_res_multifamily_walkup,restaurant,business,Year_Month
0,2016-11-01,MN01,1760,55988,79.282161,100.0,4.35,31.68,0.17,0.63,3295,683,2016-11
1,2016-11-01,MN02,5375,72165,100.324753,98.7,3.55,9.63,4.47,10.26,1271,1013,2016-11
2,2016-11-01,MN03,4708,94059,58.236802,93.4,14.95,11.15,0.32,8.66,580,947,2016-11
3,2016-11-01,MN04,3923,100090,105.402387,100.0,2.81,11.75,0.88,7.68,964,1014,2016-11
4,2016-11-01,MN05,3190,95425,516.066825,100.0,1.59,6.49,0.09,0.67,270,2083,2016-11


### 311

In [21]:
df311.head()

,Unnamed: 0,CD,Year_Month,Year,Month,Count
0,0,BK01,2016-11,2016,11,50
1,1,BK01,2016-12,2016,12,36
2,2,BK01,2017-01,2017,1,51
3,3,BK01,2017-02,2017,2,61
4,4,BK01,2017-03,2017,3,67


In [22]:
df311['Year_Month'] = pd.to_datetime(ses['month']).dt.to_period('M')
df311.dtypes

Unnamed: 0        int64
CD               object
Year_Month    period[M]
Year              int64
Month             int64
Count             int64
dtype: object

In [23]:
df311.rename({'Count':'Count311'},axis=1,inplace=True)

In [24]:
df311 = df311.drop(['Unnamed: 0'],axis =1)

In [25]:
df311.shape

(3523, 5)

In [26]:
df311.head()

,CD,Year_Month,Year,Month,Count311
0,BK01,2016-11,2016,11,50
1,BK01,2016-11,2016,12,36
2,BK01,2016-11,2017,1,51
3,BK01,2016-11,2017,2,61
4,BK01,2016-11,2017,3,67


## MERGING

In [27]:
merge1 = swm.merge(df311, how='left', left_on=['Year_Month', 'CD'], right_on = ['Year_Month', 'CD'],indicator=True)

In [ ]:
#merge1 = pd.merge(swm, df311,  how='left', left_on=['Year_Month', 'CD'], right_on = ['Year_Month', 'CD'],indicator=True)

In [30]:
#merge1 = pd.merge(swm, df311, on=['Year_Month', 'CD'], how='left', indicator = True)
merge1.head()

,CD,Year_x,Month_x,REFUSETONSCOLLECTED,PAPERTONSCOLLECTED,MGPTONSCOLLECTED,RESORGANICSTONS,fs_openhours,fs_open,trs_permits,...,seasonal_wire_litter_basket,highend_basket,publicspacereceptacle,wire_basket,total_litter_baskets,Year_Month,Year_y,Month_y,Count311,_merge
0,BK01,2016,11,5250.4,527.8,402.8,16.9,333.25,6.0,14.0,...,0.0,2.0,1.0,10.0,14.0,2016-11,2016.0,11.0,50.0,both
1,BK01,2016,11,5250.4,527.8,402.8,16.9,333.25,6.0,14.0,...,0.0,2.0,1.0,10.0,14.0,2016-11,2016.0,12.0,36.0,both
2,BK01,2016,11,5250.4,527.8,402.8,16.9,333.25,6.0,14.0,...,0.0,2.0,1.0,10.0,14.0,2016-11,2017.0,1.0,51.0,both
3,BK01,2016,11,5250.4,527.8,402.8,16.9,333.25,6.0,14.0,...,0.0,2.0,1.0,10.0,14.0,2016-11,2017.0,2.0,61.0,both
4,BK01,2016,11,5250.4,527.8,402.8,16.9,333.25,6.0,14.0,...,0.0,2.0,1.0,10.0,14.0,2016-11,2017.0,3.0,67.0,both


In [29]:
merge1.shape

(6945, 30)

In [31]:
merge2 = pd.merge(merge1, ses, on=['Year_Month', 'CD'], how='left')
merge2.head()

,CD,Year_x,Month_x,REFUSETONSCOLLECTED,PAPERTONSCOLLECTED,MGPTONSCOLLECTED,RESORGANICSTONS,fs_openhours,fs_open,trs_permits,...,over65_rate,under18_rate,pct_white_nh,poverty_rate,pct_hh_rent_burd,mean_commute,unemployment,crime_per_1000,pop_density,month
0,BK01,2016,11,5250.4,527.8,402.8,16.9,333.25,6.0,14.0,...,9.3,24.0,61.3,15.4,40.7,34.3,4.0,10.9,36626.118079,2016-11-01
1,BK01,2016,11,5250.4,527.8,402.8,16.9,333.25,6.0,14.0,...,9.3,24.0,61.3,15.4,40.7,34.3,4.0,10.9,36626.118079,2016-11-01
2,BK01,2016,11,5250.4,527.8,402.8,16.9,333.25,6.0,14.0,...,9.3,24.0,61.3,15.4,40.7,34.3,4.0,10.9,36626.118079,2016-11-01
3,BK01,2016,11,5250.4,527.8,402.8,16.9,333.25,6.0,14.0,...,9.3,24.0,61.3,15.4,40.7,34.3,4.0,10.9,36626.118079,2016-11-01
4,BK01,2016,11,5250.4,527.8,402.8,16.9,333.25,6.0,14.0,...,9.3,24.0,61.3,15.4,40.7,34.3,4.0,10.9,36626.118079,2016-11-01


In [32]:
merge3 = pd.merge(merge2, lu, on=['Year_Month', 'CD'], how='left')
merge3.head()

,CD,Year_x,Month_x,REFUSETONSCOLLECTED,PAPERTONSCOLLECTED,MGPTONSCOLLECTED,RESORGANICSTONS,fs_openhours,fs_open,trs_permits,...,numbldgs,unitstotal,asland_sqft,pct_clean_strts,pct_lot_area_open_space,pct_lot_area_public_facility_institution,pct_lot_area_res_1_2_family_bldg,pct_lot_area_res_multifamily_walkup,restaurant,business
0,BK01,2016,11,5250.4,527.8,402.8,16.9,333.25,6.0,14.0,...,25111.0,97606.0,10.58202,98.0,6.02,16.86,14.19,15.18,2577.0,1408.0
1,BK01,2016,11,5250.4,527.8,402.8,16.9,333.25,6.0,14.0,...,25111.0,97606.0,10.58202,98.0,6.02,16.86,14.19,15.18,2577.0,1408.0
2,BK01,2016,11,5250.4,527.8,402.8,16.9,333.25,6.0,14.0,...,25111.0,97606.0,10.58202,98.0,6.02,16.86,14.19,15.18,2577.0,1408.0
3,BK01,2016,11,5250.4,527.8,402.8,16.9,333.25,6.0,14.0,...,25111.0,97606.0,10.58202,98.0,6.02,16.86,14.19,15.18,2577.0,1408.0
4,BK01,2016,11,5250.4,527.8,402.8,16.9,333.25,6.0,14.0,...,25111.0,97606.0,10.58202,98.0,6.02,16.86,14.19,15.18,2577.0,1408.0


In [33]:
merge3.columns

Index(['CD', 'Year_x', 'Month_x', 'REFUSETONSCOLLECTED', 'PAPERTONSCOLLECTED',
       'MGPTONSCOLLECTED', 'RESORGANICSTONS', 'fs_openhours', 'fs_open',
       'trs_permits', 'paper_baskets', 'mgp_baskets',
       'total_number_of_households', 'hh_compost_num', 'hh_compost_percent',
       'shape_area', 'wfr_avg_recycling', 'wfr_avg_refuse', 'wfr_avg_bulk',
       'bigbelly', 'seasonal_wire_litter_basket', 'highend_basket',
       'publicspacereceptacle', 'wire_basket', 'total_litter_baskets',
       'Year_Month', 'Year_y', 'Month_y', 'Count311', '_merge', 'pct_bach_deg',
       'over65_rate', 'under18_rate', 'pct_white_nh', 'poverty_rate',
       'pct_hh_rent_burd', 'mean_commute', 'unemployment', 'crime_per_1000',
       'pop_density', 'month_x', 'month_y', 'numbldgs', 'unitstotal',
       'asland_sqft', 'pct_clean_strts', 'pct_lot_area_open_space',
       'pct_lot_area_public_facility_institution',
       'pct_lot_area_res_1_2_family_bldg',
       'pct_lot_area_res_multifamily_walkup

In [34]:
merge_final = merge3.drop(['Year_x','Month_x','Year_y','Month_y','month_x','month_y'],axis =1)

In [35]:
merge_final.head()

,CD,REFUSETONSCOLLECTED,PAPERTONSCOLLECTED,MGPTONSCOLLECTED,RESORGANICSTONS,fs_openhours,fs_open,trs_permits,paper_baskets,mgp_baskets,...,numbldgs,unitstotal,asland_sqft,pct_clean_strts,pct_lot_area_open_space,pct_lot_area_public_facility_institution,pct_lot_area_res_1_2_family_bldg,pct_lot_area_res_multifamily_walkup,restaurant,business
0,BK01,5250.4,527.8,402.8,16.9,333.25,6.0,14.0,9.0,9.0,...,25111.0,97606.0,10.58202,98.0,6.02,16.86,14.19,15.18,2577.0,1408.0
1,BK01,5250.4,527.8,402.8,16.9,333.25,6.0,14.0,9.0,9.0,...,25111.0,97606.0,10.58202,98.0,6.02,16.86,14.19,15.18,2577.0,1408.0
2,BK01,5250.4,527.8,402.8,16.9,333.25,6.0,14.0,9.0,9.0,...,25111.0,97606.0,10.58202,98.0,6.02,16.86,14.19,15.18,2577.0,1408.0
3,BK01,5250.4,527.8,402.8,16.9,333.25,6.0,14.0,9.0,9.0,...,25111.0,97606.0,10.58202,98.0,6.02,16.86,14.19,15.18,2577.0,1408.0
4,BK01,5250.4,527.8,402.8,16.9,333.25,6.0,14.0,9.0,9.0,...,25111.0,97606.0,10.58202,98.0,6.02,16.86,14.19,15.18,2577.0,1408.0


In [36]:
first_column = merge_final.pop('Year_Month')
merge_final.insert(0, 'Year_Month', first_column)

In [37]:
merge_final.head()

,Year_Month,CD,REFUSETONSCOLLECTED,PAPERTONSCOLLECTED,MGPTONSCOLLECTED,RESORGANICSTONS,fs_openhours,fs_open,trs_permits,paper_baskets,...,numbldgs,unitstotal,asland_sqft,pct_clean_strts,pct_lot_area_open_space,pct_lot_area_public_facility_institution,pct_lot_area_res_1_2_family_bldg,pct_lot_area_res_multifamily_walkup,restaurant,business
0,2016-11,BK01,5250.4,527.8,402.8,16.9,333.25,6.0,14.0,9.0,...,25111.0,97606.0,10.58202,98.0,6.02,16.86,14.19,15.18,2577.0,1408.0
1,2016-11,BK01,5250.4,527.8,402.8,16.9,333.25,6.0,14.0,9.0,...,25111.0,97606.0,10.58202,98.0,6.02,16.86,14.19,15.18,2577.0,1408.0
2,2016-11,BK01,5250.4,527.8,402.8,16.9,333.25,6.0,14.0,9.0,...,25111.0,97606.0,10.58202,98.0,6.02,16.86,14.19,15.18,2577.0,1408.0
3,2016-11,BK01,5250.4,527.8,402.8,16.9,333.25,6.0,14.0,9.0,...,25111.0,97606.0,10.58202,98.0,6.02,16.86,14.19,15.18,2577.0,1408.0
4,2016-11,BK01,5250.4,527.8,402.8,16.9,333.25,6.0,14.0,9.0,...,25111.0,97606.0,10.58202,98.0,6.02,16.86,14.19,15.18,2577.0,1408.0


In [38]:
merge_final.shape

(6945, 46)